## Travel time analysis

In [21]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
import numpy as np

### Data reading

In [23]:
# Read in travel time results and SourceSink data
scenarios = range(0,5)
results = {}
for s in scenarios:
    results[s] = pd.read_csv(f"../experiments/results_scenario_{s}.csv", index_col=0)
    results[s].drop("VehicleID", axis="columns", inplace=True)
source_sinks = pd.read_csv(f"../experiments/source_data.csv", index_col=0)

In [24]:
display(results[2].head(5))
display(source_sinks.head(5))

,Travel_Time,Startpoint,Endpoint
0,9,SourceSink13022,13012
1,14,SourceSink13033,13022
2,6,SourceSink13034,13040
3,17,SourceSink13012,13023
4,6,SourceSink10000,11303


,Road,Coordinates
10000,N1,"(90.443333, 23.7060278)"
11302,N1,"(92.298083, 20.8629167)"
11303,N2,"(90.5214438, 23.7059167)"
12253,N2,"(92.0176382, 25.1570556)"
12254,N105,"(90.5466108, 23.6904163)"


### Global average travel time

In [27]:
#calculating 95% confidence intervals of the economic losses
confidence = 0.95

average = {}
low_bound = {}
high_bound = {}
interval_range = {}
economic_interval = {}

for i in scenarios:
    results_list = results[i]["Travel_Time"].tolist()
    average[i] = np.mean(results_list)
    low_bound[i], high_bound[i] = st.norm.interval(alpha=confidence, loc=average[i], scale=st.sem(results_list))
    interval_range[i] = high_bound[i] - low_bound[i]
    economic_interval[i] = interval_range[i] / average[i] * 100

    print(f'Average travel time (95% confidence interval) for scenario {i}: {average[i]:.3f} ({low_bound[i]:.3f}, {high_bound[i]:.3f}), economic interval: {economic_interval[i]:.3f}%')

df = pd.DataFrame({
    "Average (min)": average,
    "Low bound (min)": low_bound,
    "High bound (min)": high_bound,
    "Interval range (min)": interval_range,
    "Economic interval (%)": economic_interval})
df.to_csv("../results/travel_times.csv", index_label="Scenario")
df

Average travel time (95% confidence interval) for scenario 0: 322.674 (318.753, 326.596), economic interval: 2.430%
Average travel time (95% confidence interval) for scenario 1: 322.323 (318.401, 326.245), economic interval: 2.434%
Average travel time (95% confidence interval) for scenario 2: 322.337 (318.451, 326.224), economic interval: 2.411%
Average travel time (95% confidence interval) for scenario 3: 322.884 (318.974, 326.795), economic interval: 2.422%
Average travel time (95% confidence interval) for scenario 4: 321.595 (317.701, 325.489), economic interval: 2.422%


,Average (min),Low bound (min),High bound (min),Interval range (min),Economic interval (%)
0,322.674444,318.753155,326.595733,7.842577,2.430492
1,322.322671,318.400677,326.244666,7.843989,2.433583
2,322.337481,318.451440,326.223521,7.772082,2.411163
3,322.884494,318.973688,326.795301,7.821613,2.422418
4,321.594588,317.700647,325.488529,7.787882,2.421646


### Average travel time per route